In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from scipy import sparse
import statsmodels.api as sm
from sklearn.pipeline import make_pipeline, Pipeline

In [64]:
df_sp = pd.read_csv('HistoricalPrices1978.csv')
df_sp

,Date,Open,High,Low,Close
0,05/11/22,3990.08,4049.09,3928.82,3935.18
1,05/10/22,4035.18,4068.82,3958.17,4001.05
2,05/09/22,4081.27,4081.27,3975.48,3991.24
3,05/06/22,4128.17,4157.69,4067.91,4123.34
4,05/05/22,4270.43,4270.43,4106.01,4146.87
...,...,...,...,...,...
11181,01/09/78,90.64,91.52,90.01,90.64
11182,01/06/78,91.62,92.66,91.05,91.62
11183,01/05/78,92.74,94.53,92.51,92.74
11184,01/04/78,93.52,94.10,93.16,93.52


In [65]:
df_sp[["day", "month", "year"]] = df_sp["Date"].str.split("/", expand = True)
df_sp.head()

,Date,Open,High,Low,Close,day,month,year
0,05/11/22,3990.08,4049.09,3928.82,3935.18,05,11,22
1,05/10/22,4035.18,4068.82,3958.17,4001.05,05,10,22
2,05/09/22,4081.27,4081.27,3975.48,3991.24,05,09,22
3,05/06/22,4128.17,4157.69,4067.91,4123.34,05,06,22
4,05/05/22,4270.43,4270.43,4106.01,4146.87,05,05,22


In [66]:
df_sp = df_sp.rename(columns={'Date':'date', ' Open':'open', ' High':'high', ' Low':'low',
                             ' Close':'close'})
df_sp.head()

,date,open,high,low,close,day,month,year
0,05/11/22,3990.08,4049.09,3928.82,3935.18,05,11,22
1,05/10/22,4035.18,4068.82,3958.17,4001.05,05,10,22
2,05/09/22,4081.27,4081.27,3975.48,3991.24,05,09,22
3,05/06/22,4128.17,4157.69,4067.91,4123.34,05,06,22
4,05/05/22,4270.43,4270.43,4106.01,4146.87,05,05,22


In [67]:
df_sp['avg_open_yr'] = df_sp.groupby(df_sp['year'])['open'].transform('mean')
df_sp['avg_high_yr'] = df_sp.groupby(df_sp['year'])['high'].transform('mean')
df_sp['avg_low_yr'] = df_sp.groupby(df_sp['year'])['low'].transform('mean')
df_sp.head()

,date,open,high,low,close,day,month,year,avg_open_yr,avg_high_yr,avg_low_yr
0,05/11/22,3990.08,4049.09,3928.82,3935.18,05,11,22,4423.340444,4458.361111,4374.597222
1,05/10/22,4035.18,4068.82,3958.17,4001.05,05,10,22,4423.340444,4458.361111,4374.597222
2,05/09/22,4081.27,4081.27,3975.48,3991.24,05,09,22,4423.340444,4458.361111,4374.597222
3,05/06/22,4128.17,4157.69,4067.91,4123.34,05,06,22,4423.340444,4458.361111,4374.597222
4,05/05/22,4270.43,4270.43,4106.01,4146.87,05,05,22,4423.340444,4458.361111,4374.597222


In [68]:
df_sp_yr = df_sp[['year', 'avg_open_yr', 'avg_high_yr', 'avg_low_yr']]
df_sp_yr.head()

,year,avg_open_yr,avg_high_yr,avg_low_yr
0,22,4423.340444,4458.361111,4374.597222
1,22,4423.340444,4458.361111,4374.597222
2,22,4423.340444,4458.361111,4374.597222
3,22,4423.340444,4458.361111,4374.597222
4,22,4423.340444,4458.361111,4374.597222


In [72]:
df_sp_yr_unique = df_sp_yr.drop_duplicates()
df_sp_yr_unique

,year,avg_open_yr,avg_high_yr,avg_low_yr
0,22,4423.340444,4458.361111,4374.597222
90,21,4271.170714,4291.877579,4251.042698
342,20,3217.750277,3242.183953,3190.484466
595,19,2911.627619,2923.622341,2899.439921
847,18,2747.784900,2762.655418,2730.275259
1098,17,2448.275896,2454.344821,2441.850677
1349,16,2094.091548,2103.306865,2083.753135
1601,15,2061.268016,2071.863095,2049.309524
1853,14,1930.754484,1938.835595,1922.292976
2105,13,1642.295516,1649.615714,1635.737024


In [73]:
df_funds = pd.read_csv('buyouts_funds/buyouts_funds_csv/buyouts_funds_final.csv')
df_funds.head()

,Unnamed: 0,fund_id,fund_name,fm_id,strategy,sector,region,country,target_fund_size_amt,target_fund_size_currency,fund_size_amt,fund_currency,status,stage,vintage_year,open_date
0,0,34866,Ascent Venture Partners VII,6381,Venture Capital / Growth Equity,TMT,North America,United States,150000000.0,USD,NaN,NaN,Launched,NaN,2020,2020-0
1,1,36284,Fall Line Endurance Fund (Open-Ended),15168,Venture Capital / Growth Equity,TMT,North America,United States,NaN,NaN,285000000.0,USD,Currently Investing,NaN,2019,2016-7
2,2,36284,Fall Line Endurance Fund (Open-Ended),15168,Venture Capital / Growth Equity,Agribusiness,North America,United States,NaN,NaN,285000000.0,USD,Currently Investing,NaN,2019,2016-7
3,3,39769,Star Mountain U.S. LMM Secondary Fund II,1671,Secondaries,Diversified,North America,United States,500000000.0,USD,200000000.0,USD,Currently Investing,1st Close,2020,2020-12
4,4,43372,Waterfall Sentinel Fund I,19136,Buyout / Corporate Private Equity,Diversified,North America,United States,300000000.0,USD,80000000.0,USD,Currently Investing,1st Close,2019,2019-1
